In [1]:
import tensorflow as tf

import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

# def pipe(data, batch_size = 128, shuffle = False):
#     dataset = tf.data.Dataset.from_tensor_slices(data)
#     if shuffle:
#         dataset = dataset.shuffle(buffer_size = batch_size * 10)
#     dataset = dataset.batch(batch_size)
#     #dataset = dataset.prefetch((batch_size * 2) + 1)
#     dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
#     return dataset

# (tr_x, tr_y), (te_x, te_y) = tf.keras.datasets.cifar10.load_data()



In [2]:
# tr_x = tr_x * 1/255
# te_x = te_x * 1/255

# batch_size = 128

# tr_data = pipe((tr_x, tr_y), batch_size = batch_size, shuffle = True)
# te_data = pipe((te_x, te_y), batch_size = batch_size, shuffle = False)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd '/content/drive/MyDrive/DATA VISUALIZATION/2016_dataset/gaf_images'

/content/drive/.shortcut-targets-by-id/1psvNNdzjOZtPf9i4b7gJzkYmUTZL7w5O/DATA VISUALIZATION/2016_dataset/gaf_images


In [5]:
import pandas as pd
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import matplotlib.image as mpimg
from  matplotlib import pyplot as plt
%matplotlib inline

In [6]:
IMG_WIDTH=64
IMG_HEIGHT=64
batch_size=1

train_dir = r'/content/drive/MyDrive/DATA VISUALIZATION/2016_dataset/gaf_images/Image_data/gasf_split/train'
test_dir =  r'/content/drive/MyDrive/DATA VISUALIZATION/2016_dataset/gaf_images/Image_data/gasf_split/test'
val_dir =  r'/content/drive/MyDrive/DATA VISUALIZATION/2016_dataset/gaf_images/Image_data/gasf_split/val'

In [7]:
image_gen_train = ImageDataGenerator(rescale=1./255,
                                     zoom_range=0.2, 
                                     rotation_range=65,
                                     shear_range=0.09,
                                     horizontal_flip=True,
                                     vertical_flip=True)

In [8]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='sparse')

Found 2258 images belonging to 2 classes.


In [9]:
image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=val_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='sparse')

Found 484 images belonging to 2 classes.


In [10]:
train_data_gen.class_indices.keys()

dict_keys(['abnormal', 'normal'])

In [11]:
!pip install einops

In [12]:
def pipe(data, batch_size = 128, shuffle = False):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    if shuffle:
        dataset = dataset.shuffle(buffer_size = batch_size * 10)
    dataset = dataset.batch(batch_size)
    #dataset = dataset.prefetch((batch_size * 2) + 1)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

In [13]:
# tr_data = pipe((train_data_gen, train_data_gen.labels), batch_size = batch_size, shuffle = True)
# val_data = pipe((val_data_gen, val_data_gen.labels), batch_size = batch_size, shuffle = False)

In [14]:
import coatnet

model = coatnet.coatnet7(input_shape = (64, 64, 3), include_top = False,classes=2)

flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
drop_out = tf.keras.layers.Dropout(0.5)(flatten)
dense = tf.keras.layers.Dense(2048, activation = "relu")(drop_out)
prediction = tf.keras.layers.Dense(10, activation = "softmax", name = "prediction")(dense)
model = tf.keras.Model(model.input, prediction)

In [15]:
loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(1e-4)
metric = [tf.keras.metrics.sparse_categorical_accuracy]
model.compile(loss = loss, optimizer = opt, metrics = metric)

In [16]:
model.fit(train_data_gen, validation_data = val_data_gen, epochs = 10)

Epoch 1/10
2258/2258 [==============================] - 886s 347ms/step - loss: 0.6525 - sparse_categorical_accuracy: 0.7710 - val_loss: 0.7523 - val_sparse_categorical_accuracy: 0.7975
Epoch 2/10
2258/2258 [==============================] - 765s 339ms/step - loss: 0.5445 - sparse_categorical_accuracy: 0.7927 - val_loss: 0.5054 - val_sparse_categorical_accuracy: 0.7975
Epoch 3/10
2258/2258 [==============================] - 752s 333ms/step - loss: 0.5220 - sparse_categorical_accuracy: 0.7972 - val_loss: 0.5214 - val_sparse_categorical_accuracy: 0.7975
Epoch 4/10
2258/2258 [==============================] - 743s 329ms/step - loss: 0.5307 - sparse_categorical_accuracy: 0.7954 - val_loss: 0.5045 - val_sparse_categorical_accuracy: 0.7975
Epoch 5/10
2258/2258 [==============================] - 744s 329ms/step - loss: 0.5147 - sparse_categorical_accuracy: 0.7972 - val_loss: 0.5561 - val_sparse_categorical_accuracy: 0.7975
Epoch 6/10
2258/2258 [==============================] - 740s 328ms/ste

In [17]:
model.save_weights("model.h5")

In [18]:
model.load_weights("model.h5")

In [ ]:
image_gen_test = ImageDataGenerator(rescale=1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
                                                 directory=val_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='sparse')

In [20]:
loss = tf.keras.losses.sparse_categorical_crossentropy
metric = [tf.keras.metrics.sparse_categorical_accuracy]
model.compile(loss = loss, metrics = metric)
model.evaluate(test_data_gen)

484/484 [==============================] - 105s 148ms/step - loss: 0.5104 - sparse_categorical_accuracy: 0.7975


[0.5104355812072754, 0.797520637512207]